In [87]:
#Cargamos las librerías necesarias para el análisis
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import time 
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.io import loadmat

In [88]:
from scipy.io import loadmat
import os
from pathlib import Path
from glob import glob
import pathlib
from scipy.io import loadmat
data=[]
wdata=[]
i=0
contador=0
current_dir = '../data/Francis_data/Francis_data//Baseline_fmri/HC'
group=1 # los grupos están organizados por carpetas. en este caso HC = grupo 1
all_sub_dir_paths = glob(str(current_dir) + '/*/') 
feat=[]
for file in all_sub_dir_paths:
    i=0
    for file in sorted(glob(file + '/*.mat')):
        dat=loadmat(file)
        p=dat['S']
        thePoints=dat['S']['thePoints'][0][0]
        Sbuttons=dat['S']['Sbuttons']
        Ts=dat['S']['Ts'][0][0][0]
        trial=dat['S']['trial_cond'][0][0][i][0]
        length=len(Sbuttons[0][0])
        rule = Ts[-1]/length
        time = np.transpose(np.arange(start=0, stop=Ts[-1]+rule, step=rule))
        if time.shape[0] != thePoints.shape[0]:
            time = time[0:thePoints.shape[0]]
        data=[time, thePoints[:,0], thePoints[:,1], trial,group]
        data=np.transpose(data)
    #    data = data[(data[1] != 683) & (data[2] != 384)]
    #    df=pd.DataFrame.transpose(pd.DataFrame(data))
        wdata.append(data)
        i=i+1
        data=[]
    #    print(file,i)


/Users/carloshumbertomenesesmurillo/opt/anaconda3/envs/geo_env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)


In [89]:
current_dir = '../data/Francis_data/Francis_data//Baseline_fmri/Left_MAS'
group=2 # los grupos están organizados por carpetas. en este caso Left_MAS = grupo 2
all_sub_dir_paths = glob(str(current_dir) + '/*/')
for file in all_sub_dir_paths:
    i=0
    for file in sorted(glob(file + '/*.mat')):
        dat=loadmat(file)
        p=dat['S']
        thePoints=dat['S']['thePoints'][0][0]
        Sbuttons=dat['S']['Sbuttons']
        Ts=dat['S']['Ts'][0][0][0]
        trial=dat['S']['trial_cond'][0][0][i][0]
        length=len(Sbuttons[0][0])
        rule = Ts[-1]/length
        time = np.transpose(np.arange(start=0, stop=Ts[-1]+rule, step=rule))
        if time.shape[0] != thePoints.shape[0]:
            time = time[0:thePoints.shape[0]]
        data=[time, thePoints[:,0], thePoints[:,1], trial,group]
        data=np.transpose(data)
    #    data = data[(data[1] != 683) & (data[2] != 384)]
    #    df=pd.DataFrame.transpose(pd.DataFrame(data))
        wdata.append(data)
        i=i+1
        data=[]
    #    print(file,i)


In [90]:
current_dir = '../data/Francis_data/Francis_data//Baseline_fmri/Right_MAS'
group=0 # los grupos están organizados por carpetas. en este caso Right_MAS = grupo 0
all_sub_dir_paths = glob(str(current_dir) + '/*/')
for file in all_sub_dir_paths:
    i=0
    for file in sorted(glob(file + '/*.mat')):
        dat=loadmat(file)
        p=dat['S']
        thePoints=dat['S']['thePoints'][0][0]
        Sbuttons=dat['S']['Sbuttons']
        Ts=dat['S']['Ts'][0][0][0]
        trial=dat['S']['trial_cond'][0][0][i][0]
        length=len(Sbuttons[0][0])
        rule = Ts[-1]/length
        time = np.transpose(np.arange(start=0, stop=Ts[-1]+rule, step=rule))
        if time.shape[0] != thePoints.shape[0]:
            time = time[0:thePoints.shape[0]]
        data=[time, thePoints[:,0], thePoints[:,1], trial,group]
        data=np.transpose(data)
    #    data = data[(data[1] != 683) & (data[2] != 384)]
    #    df=pd.DataFrame.transpose(pd.DataFrame(data))
        wdata.append(data)
        i=i+1
        data=[]
    #    print(file,i)

In [117]:
data=pd.DataFrame(wdata)
data.rename(columns={0: "time", 1: "X", 2: 'Y', 3: 'event',4:"group"}, inplace=True)
data["time"][3][4]

0.04000022792321036

In [119]:
df = pd.DataFrame(data)

# Configurar la opción para evitar la notación científica
pd.set_option('display.float_format', '{:.17f}'.format)
pd.set_option('display.max_colwidth', -1)
# Guardar el DataFrame en un archivo CSV
df.to_csv('./data.csv', index=False)

# Restaurar la configuración original si es necesario
pd.reset_option('display.float_format')

In [129]:
#cargar csv
df=pd.read_csv('./data.csv',index_col=False)
df["time"][4]

'[0.         0.01000107 0.02000213 0.0300032  0.04000426 0.05000533\n 0.0600064  0.07000746 0.08000853 0.09000959 0.10001066 0.11001173\n 0.12001279 0.13001386 0.14001492 0.15001599 0.16001705 0.17001812\n 0.18001919 0.19002025 0.20002132 0.21002238 0.22002345 0.23002452\n 0.24002558 0.25002665 0.26002771 0.27002878 0.28002985 0.29003091\n 0.30003198 0.31003304 0.32003411 0.33003518 0.34003624 0.35003731\n 0.36003837 0.37003944 0.38004051 0.39004157 0.40004264 0.4100437\n 0.42004477 0.43004583 0.4400469  0.45004797 0.46004903 0.4700501\n 0.48005116 0.49005223 0.5000533  0.51005436 0.52005543 0.53005649\n 0.54005756 0.55005863 0.56005969 0.57006076 0.58006182 0.59006289\n 0.60006396 0.61006502 0.62006609 0.63006715 0.64006822 0.65006929\n 0.66007035 0.67007142 0.68007248 0.69007355 0.70007461 0.71007568\n 0.72007675 0.73007781 0.74007888 0.75007994 0.76008101 0.77008208\n 0.78008314 0.79008421 0.80008527 0.81008634 0.82008741 0.83008847\n 0.84008954 0.8500906  0.86009167 0.87009274 0.88

In [130]:
#Contar cuantos hay de grupo 2,0 y 1
df.groupby('group').count()

,time,X,Y,event
group,,,,
0,920,920,920,920
1,1160,1160,1160,1160
2,600,600,600,600


In [131]:
#Agrupar por evento
df.groupby('event').count()

,time,X,Y,group
event,,,,
1,670,670,670,670
2,670,670,670,670
3,670,670,670,670
4,670,670,670,670
